# see the situation :

In [7]:
%echo "Total: $(du -ch *.log | grep total)"

UsageError: Line magic function `%echo` not found.


In [9]:
import os
from pathlib import Path

def get_size(path):
    """Calculate size of a file or directory (recursive) in bytes"""
    if path.is_file():
        return path.stat().st_size
    return sum(f.stat().st_size for f in path.glob('**/*') if f.is_file())

def list_sizes(target_dir='.', sort_by='size', reverse=True, human_readable=True, 
              include_hidden=False, max_depth=None, unit='auto'):
    """
    List sizes of files and directories with formatting options.
    
    Args:
        target_dir (str): Directory path (default: current dir)
        sort_by (str): 'size', 'name', or 'none'
        reverse (bool): Sort descending (default: True)
        human_readable (bool): Format sizes (default: True)
        include_hidden (bool): Show hidden items (default: False)
        max_depth (int): None=recursive, 0=current dir only, 1=+1 subdir level
        unit (str): 'auto'(smart), 'B', 'KB', 'MB', 'GB', 'TB' (forced unit)
    
    Returns:
        list: [(path, size_str), ...] sorted tuples
    """
    target = Path(target_dir)
    if not target.exists():
        raise FileNotFoundError(f"Directory not found: {target_dir}")
    
    items = []
    for item in target.iterdir():
        if not include_hidden and item.name.startswith('.'):
            continue
            
        if max_depth is None or max_depth >= 0:
            size = get_size(item)
            items.append((item, size))
    
    # Sorting
    if sort_by == 'size':
        items.sort(key=lambda x: x[1], reverse=reverse)
    elif sort_by == 'name':
        items.sort(key=lambda x: x[0].name, reverse=reverse)
    
    # Formatting
    def format_size(bytes_size):
        if not human_readable:
            return str(bytes_size)
        
        units = ['B', 'KB', 'MB', 'GB', 'TB']
        if unit != 'auto':
            units = [unit]
        
        size = float(bytes_size)
        unit_idx = 0
        
        while size >= 1024 and unit_idx < len(units)-1 and units[unit_idx] != unit:
            size /= 1024
            unit_idx += 1
            
        return f"{size:.2f} {units[unit_idx]}".rstrip('B') + units[unit_idx]
    
    return [(str(item[0]), format_size(item[1])) for item in items]

# Example usage
if __name__ == "__main__":
    print(f"{'Path':<50} {'Size':>15}")
    print('-' * 65)
    for path, size in list_sizes(max_depth=1, unit='MB'):
        print(f"{path:<50} {size:>15}")

Path                                                          Size
-----------------------------------------------------------------
minicpmv_o_2_6_model                               17432640217.00 MMB
my_model                                           16211274105.00 MMB
llama.cpp                                          253349403.00 MMB
MiniCPM_lightning.ipynb                              292800.00 MMB
232-modele-cv-francais.docx                          195967.00 MMB
1131w-uHRaEYx8dVI (1).webp                           119898.00 MMB
cv.webp                                              119898.00 MMB
232-modele-cv-francais.pdf                           116536.00 MMB
cv.pdf                                               116536.00 MMB
cv2.path                                             116536.00 MMB
cv2.pdf                                              116536.00 MMB
promt.ipynb                                           52867.00 MMB
nohup.out                                             37

In [1]:
def check_pytorch_installation():
    import torch
    import pkg_resources
    import sys
    import platform
    import subprocess
    from typing import Optional

    def run_command(cmd: str) -> Optional[str]:
        try:
            return subprocess.check_output(cmd, shell=True, stderr=subprocess.PIPE).decode().strip()
        except:
            return None

    # --- Python and System Info ---
    print("\n" + "="*50)
    print("Python Environment Details")
    print("="*50)
    print(f"Python version: {sys.version}")
    print(f"Python executable: {sys.executable}")
    print(f"Platform: {platform.platform()}")

    # --- PyTorch Core Checks ---
    print("\n" + "="*50)
    print("PyTorch Installation Details")
    print("="*50)
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")

    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"cuDNN version: {torch.backends.cudnn.version()}")
        print(f"Current device: {torch.cuda.current_device()}")
        print(f"Device name: {torch.cuda.get_device_name(0)}")

    # --- Critical Flash Attention Checks ---
    print("\n" + "="*50)
    print("Flash Attention Prerequisites")
    print("="*50)

    # 1. Check CUDA Toolkit (nvcc)
    nvcc_version = run_command("nvcc --version | grep 'release' | awk '{print $6}'")
    print(f"CUDA Toolkit (nvcc): {nvcc_version if nvcc_version else 'NOT FOUND'}")

    # 2. Check cuDNN (if CUDA is available)
    if torch.cuda.is_available():
        cudnn_version = run_command("grep -m1 CUDNN_MAJOR -A2 /usr/include/cudnn_version.h 2>/dev/null || echo 'NOT FOUND'")
        print(f"cuDNN headers: {'Found' if 'CUDNN_MAJOR' in str(cudnn_version) else 'NOT FOUND'}")

    # 3. Check build tools (gcc, cmake, ninja)
    print("\nBuild Tools:")
    print(f"gcc: {run_command('gcc --version | head -n1') or 'NOT FOUND'}")
    print(f"cmake: {run_command('cmake --version | head -n1') or 'NOT FOUND'}")
    print(f"ninja: {run_command('ninja --version') or 'NOT FOUND'}")

    # --- Package Versions ---
    print("\n" + "="*50)
    print("Related Package Versions")
    print("="*50)
    packages = [
        'torch', 'torchvision', 'torchaudio', 
        'flash-attn', 'einops', 'transformers',
        'numpy', 'pillow'
    ]
    for pkg in packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            print(f"{pkg}: {version}")
        except:
            print(f"{pkg}: Not installed")

    # --- CUDA Toolkit Components ---
    try:
        print("\n" + "="*50)
        print("CUDA Toolkit Components")
        print("="*50)
        cuda_pkgs = [
            'nvidia-cuda-runtime-cu12', 'nvidia-cudnn-cu12',
            'nvidia-nccl-cu12', 'nvidia-cublas-cu12'
        ]
        for pkg in cuda_pkgs:
            try:
                version = pkg_resources.get_distribution(pkg).version
                print(f"{pkg}: {version}")
            except:
                print(f"{pkg}: Not found")
    except Exception as e:
        print(f"\nError checking CUDA packages: {e}")

    print("\n" + "="*50)
    print("Environment Check Complete")
    print("="*50)

# Run the check
check_pytorch_installation()

/tmp/ipykernel_1729/3099177714.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources



Python Environment Details
Python version: 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
Python executable: /home/zeus/miniconda3/envs/cloudspace/bin/python
Platform: Linux-6.8.0-1031-gcp-x86_64-with-glibc2.31

PyTorch Installation Details
PyTorch version: 2.7.0+cu126
CUDA available: True
CUDA version: 12.6
cuDNN version: 90501
Current device: 0
Device name: NVIDIA L4

Flash Attention Prerequisites
CUDA Toolkit (nvcc): V12.1.105
cuDNN headers: Found

Build Tools:
gcc: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
cmake: cmake version 3.16.3
ninja: 1.11.1.git.kitware.jobserver-1

Related Package Versions
torch: 2.7.0
torchvision: 0.22.0
torchaudio: 2.7.0
flash-attn: Not installed
einops: 0.8.1
transformers: 4.52.4
numpy: 1.26.4
pillow: 11.2.1

CUDA Toolkit Components
nvidia-cuda-runtime-cu12: 12.6.77
nvidia-cudnn-cu12: 9.5.1.17
nvidia-nccl-cu12: 2.26.2
nvidia-cublas-cu12: 12.6.4.1

Environment Check Complete


## Setup :

In [2]:
#Before downloading, install ModelScope first by using the following command

!pip install modelscope

- for vllm inference ==> vllm just for PyTorch (.bin, .safetensors) not for gguf file 

In [3]:
!uv pip install vllm --torch-backend=auto

Using Python 3.10.10 environment at: /home/zeus/miniconda3/envs/cloudspace
Audited 1 package in 155ms


- test model with vllm inference to see if there s a time gain 

In [ ]:
#another way aconrding to the documentation is the same as the next one

!nohup vllm serve /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6 &

In [ ]:
!tail -n 30 nohup.out

In [ ]:
!ls /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6

## libraries : 

In [4]:
# test.py
import torch
from PIL import Image
from modelscope import AutoModel, AutoTokenizer

import json
import os
from os.path import join
import random
import requests

## MiniCPM_V_2_6 :

In [2]:
%%writefile requirements_MiniCPM_V_2_6.txt
Pillow==10.1.0
torch==2.1.2
torchvision==0.16.2
transformers==4.40.0
sentencepiece==0.1.99
decord


Writing requirements_MiniCPM_V_2_6.txt


In [ ]:
!pip install -r requirements_MiniCPM_V_2_6.txt

In [2]:
!pwd

/teamspace/studios/this_studio


## MiniCPM-o-2_6-gguf & MiniCPM-o-2_6 :

In [5]:
%%writefile requirements_MiniCPM_2_6.txt
Pillow==10.1.0
torch==2.3.1
torchaudio==2.3.1
torchvision==0.18.1
transformers==4.44.2
librosa==0.9.0
soundfile==0.12.1
vector-quantize-pytorch==1.18.5
vocos==0.1.0
decord
moviepy

Overwriting requirements_MiniCPM_2_6.txt


In [6]:
!pip install -r requirements_MiniCPM_2_6.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 70.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 179.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 137.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 90.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 115.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 52.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 107.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 115.8 MB/s eta 0:00:000

# Download Model :

## MiniCPM-V-2_6 :

In [ ]:
!modelscope download --model OpenBMB/MiniCPM-V-2_6

In [ ]:
!modelscope download --model OpenBMB/MiniCPM-V-2_6 --local_dir ./minicpmv_model

In [12]:
#Model Download
from modelscope import snapshot_download
model_dir = snapshot_download('OpenBMB/MiniCPM-V-2_6')

In [13]:
model_dir

'/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-V-2_6'

In [2]:
!mv '/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-V-2_6' '/teamspace/studios/this_studio/my_model'

In [ ]:
!ls -l

In [ ]:
# Download the full model repo

!modelscope download --model OpenBMB/MiniCPM-V-2_6

## MiniCPM-o-2_6-gguf :

In [1]:
!modelscope download --model OpenBMB/MiniCPM-o-2_6-gguf


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'


Successfully Downloaded from model OpenBMB/MiniCPM-o

In [2]:
!ls /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6-gguf

Model-7.6B-F16.gguf	Model-7.6B-Q5_0.gguf	README.md
Model-7.6B-Q4_0.gguf	Model-7.6B-Q5_1.gguf	configuration.json
Model-7.6B-Q4_1.gguf	Model-7.6B-Q5_K_M.gguf	mmproj-model-f16.gguf
Model-7.6B-Q4_K_M.gguf	Model-7.6B-Q6_K.gguf
Model-7.6B-Q4_K_S.gguf	Model-7.6B-Q8_0.gguf


## MiniCPM-o-2_6 :

In [ ]:
!modelscope download --model OpenBMB/MiniCPM-o-2_6 --local_dir ./minicpmv_o_2_6_model

: 

In [7]:
!modelscope download --model OpenBMB/MiniCPM-o-2_6


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'


Successfully Downloaded from model OpenBMB/MiniCPM-o

In [3]:
!ls /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6

README.md			  model.safetensors.index.json
added_tokens.json		  modeling_minicpmo.py
assets				  modeling_navit_siglip.py
config.json			  preprocessor_config.json
configuration.json		  processing_minicpmo.py
configuration_minicpm.py	  resampler.py
image_processing_minicpmv.py	  special_tokens_map.json
merges.txt			  tokenization_minicpmo_fast.py
model-00001-of-00004.safetensors  tokenizer.json
model-00002-of-00004.safetensors  tokenizer_config.json
model-00003-of-00004.safetensors  utils.py
model-00004-of-00004.safetensors  vocab.json


# inference : 

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.7.0+cu126
CUDA version: 12.6
CUDA available: True


## exp 1 :

In [ ]:
!pip install llama-cpp-python --prefer-binary


In [ ]:
# Ubuntu/Debian
!sudo apt update
!sudo apt install -y build-essential cmake python3-dev libopenblas-dev

# Optional: For faster compilation
!pip install ninja


In [ ]:
!sudo apt update
!sudo apt install -y libomp-dev


In [ ]:
!pip install llama-cpp-python==0.3.9


In [2]:
from llama_cpp import Llama
model_path = "/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6-gguf"
llm = Llama(model_path=model_path, n_ctx=2048)
your_image_tensor = "1131w-uHRaEYx8dVI (1).webp"
output = llm("What is in this image?", images=[your_image_tensor])


ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
from llama_cpp import Llama

llm = Llama(
      model_path="./models/7B/llama-model.gguf",
      # n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      # n_ctx=2048, # Uncomment to increase the context window
)
output = llm(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

In [ ]:
#-------------------------------the previous was just trying----------------------------------------------------------------------

## MiniCPM-V-2_6 :

In [10]:
!pwd

/teamspace/studios/this_studio


In [20]:
!pip install addict

In [ ]:
!pip install datasets

In [ ]:
# test.py
import torch
from PIL import Image
from modelscope import AutoModel, AutoTokenizer

my_model_dir = "/teamspace/studios/this_studio/my_model/MiniCPM-V-2_6"
model = AutoModel.from_pretrained(my_model_dir, trust_remote_code=True,
    attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(my_model_dir, trust_remote_code=True)

image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print(res)

## if you want to use streaming, please make sure sampling=True and stream=True
## the model.chat will return a generator
"""
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

"""

In [5]:
image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    system_prompt="you a professional OCR "
)
print(res)

The image is a resume of an individual named Célia Naudin, who has experience in communication and marketing.


the following command for verifying if a problem is in flash attention :

the previous command mention that there s a problem with flash attention , so do this command :

## OpenBMB/MiniCPM-o-2_6 :

In [1]:
# test.py
import torch
from PIL import Image
from modelscope import AutoModel, AutoTokenizer

# to see the path you can execute the download command :

my_model_dir = "/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6"

model = AutoModel.from_pretrained(
    my_model_dir,
    trust_remote_code=True,
    attn_implementation='sdpa', # sdpa or flash_attention_2
    torch_dtype=torch.bfloat16,
    init_vision=True,
    init_audio=True,
    init_tts=True
)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(my_model_dir, trust_remote_code=True)

# In addition to vision-only mode, tts processor and vocos also needs to be initialized
model.init_tts()

image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print("the response of the model is : ",res)

## if you want to use streaming, please make sure sampling=True and stream=True
## the model.chat will return a generator
"""
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

"""

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


the response of the model is :  The image is a resume (CV) for Célia Naudin. It includes her contact information, experiences, languages spoken, hobbies, education background, and skills.


'\nres = model.chat(\n    image=None,\n    msgs=msgs,\n    tokenizer=tokenizer,\n    sampling=True,\n    stream=True\n)\n\ngenerated_text = ""\nfor new_text in res:\n    generated_text += new_text\n    print(new_text, flush=True, end=\'\')\n\n'

In [2]:
image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]},
       {'role': 'system', 'content': 'you are a prfessional cv analyser, you will provide the answer in frensh language'}]

res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print("the response of the model is : ",res)

the response of the model is :  L'image montre un CV de Célia Naudin, qui occupe le poste de Chargée de communication. Le document présente ses expériences professionnelles, ses compétences linguistiques et éducatives, ses centres d'intérêt et ses compétences.

### Expériences
- **Groupe Frame (2021 - 2026)**
  - Chargée de communication
  - Gestion de la stratégie de communication
  - Développement des réseaux sociaux
  - Production de contenus éditoriaux
  - Organisation d'événements

- **Bancollect (2019 - 2020)**
  - Assistante communication en alternance
  - Création et publication de contenu
  - Participation à des projets de marketing
  - Veille sur les tendances des réseaux sociaux

### Éducation
- **École Amédé Autran (2017 - 2020)**
  - Master communication en alternance
  - Licence marketing digital

### Langues
- Français courant
- Anglais courant
- Espagnol

### Loisirs
- Mode
- Photographie
- Voyage

### Compétences
- Réseaux sociaux
- Graphisme
- Référencement
- Vidéo

L

## MiniCPM-o-2_6-gguf  :

In [11]:
%cd /teamspace/studios/this_studio/llama.cpp

/teamspace/studios/this_studio/llama.cpp


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [23]:
import os
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

Current directory: /teamspace/studios/this_studio


In [18]:
%rm -rf llama.cpp

In [15]:
%cd ..

/teamspace/studios/this_studio


In [19]:
!git clone git@github.com:OpenBMB/llama.cpp.git
%cd llama.cpp
!git checkout minicpm-omni

Cloning into 'llama.cpp'...
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


⚡️ Tip	Connect GitHub to Studios: https://lightning.ai/younessaznag/home?settings=integrations

[Errno 2] No such file or directory: 'llama.cpp'
/teamspace/studios/this_studio
fatal: not a git repository (or any parent up to mount point /teamspace/studios)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [20]:
!mkdir -p ~/.ssh
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!git clone git@github.com:OpenBMB/llama.cpp.git

# github.com:22 SSH-2.0-f27bdd3a
# github.com:22 SSH-2.0-f27bdd3a
# github.com:22 SSH-2.0-f27bdd3a
# github.com:22 SSH-2.0-f27bdd3a
# github.com:22 SSH-2.0-f27bdd3a
Cloning into 'llama.cpp'...
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


⚡️ Tip	Connect GitHub to Studios: https://lightning.ai/younessaznag/home?settings=integrations



In [21]:
!ssh-keygen -t ed25519 -C aznag_youness@yahoo.fr

Generating public/private ed25519 key pair.
Enter file in which to save the key (/home/zeus/.ssh/id_ed25519): ^C


In [22]:
!git clone git@github.com:OpenBMB/llama.cpp.git


Cloning into 'llama.cpp'...
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


⚡️ Tip	Connect GitHub to Studios: https://lightning.ai/younessaznag/home?settings=integrations



## exp 2 (cv analyser prompt) :

### prompt 1 :

In [25]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

def build_cv_prompt(job_description: str, language: str = "en") -> str:
    instructions = {
        "en": {
            "extract": "Extract the following information from the resume:",
            "fields": [
                "- Full Name",
                "- Contact Information (email, phone, LinkedIn, etc.)",
                "- Education (degree, institution, graduation year)",
                "- Work Experience (role, company, duration, responsibilities)",
                "- Skills (technical, soft, languages)",
                "- Certifications (if any)",
                "- Projects (if any)"
            ],
            "match": "Match the extracted profile against the following job description:",
            "rate": "Rate the candidate's suitability for this job on a scale from 0 to 100.",
            "justify": "Justify your rating using bullet points.",
            "summarize": "Summarize:",
            "strengths": "- Candidate's strengths",
            "weaknesses": "- Candidate's weaknesses"
        },
        "fr": {
            "extract": "Extraire les informations suivantes du CV :",
            "fields": [
                "- Nom complet",
                "- Coordonnées (email, téléphone, LinkedIn, etc.)",
                "- Formation (diplôme, établissement, année d'obtention)",
                "- Expérience professionnelle (poste, entreprise, durée, responsabilités)",
                "- Compétences (techniques, générales, langues)",
                "- Certifications (le cas échéant)",
                "- Projets (le cas échéant)"
            ],
            "match": "Comparer le profil extrait avec la description de poste suivante :",
            "rate": "Évaluer l'adéquation du candidat pour ce poste sur une échelle de 0 à 100.",
            "justify": "Justifier votre évaluation avec des points clés.",
            "summarize": "Résumer :",
            "strengths": "- Points forts du candidat",
            "weaknesses": "- Points faibles du candidat"
        },
        "es": {
            "extract": "Extraer la siguiente información del currículum:",
            "fields": [
                "- Nombre completo",
                "- Información de contacto (email, teléfono, LinkedIn, etc.)",
                "- Educación (título, institución, año de graduación)",
                "- Experiencia laboral (puesto, empresa, duración, responsabilidades)",
                "- Habilidades (técnicas, blandas, idiomas)",
                "- Certificaciones (si aplica)",
                "- Proyectos (si aplica)"
            ],
            "match": "Comparar el perfil extraído con la siguiente descripción del puesto:",
            "rate": "Calificar la idoneidad del candidato para este trabajo en una escala de 0 a 100.",
            "justify": "Justificar su calificación con puntos clave.",
            "summarize": "Resumir:",
            "strengths": "- Fortalezas del candidato",
            "weaknesses": "- Debilidades del candidato"
        }
    }
    
    # Get instructions for the specified language, default to English
    lang = instructions.get(language, instructions["en"])
    
    return f"""
You must / Vous devez / Usted debe:
1. {lang['extract']}
   {chr(10).join(lang['fields'])}

2. {lang['match']}
---
{job_description}
---

3. {lang['rate']}

4. {lang['justify']}

5. {lang['summarize']}
   {lang['strengths']}
   {lang['weaknesses']}

Language of response: {language.upper()}
"""

In [26]:

def analyze_cv(cv_image_path: str, job_description: str, language: str):


    # Load image
    image = Image.open(cv_image_path).convert("RGB")

    # Build prompt
    prompt = build_cv_prompt(job_description, language)

    msgs = [{'role': 'user', 'content': [image, prompt]},
            {'role': 'system', 
             'content': f"You are a professional HR assistant AI that can analyze resumes from image input using OCR and answering in {language}."}
            ]

    response = model.chat(
        image=None,
        msgs=msgs,
        tokenizer=tokenizer
    )


    return response





In [27]:
from datetime import datetime

job = """Looking for a data analyst with 3+ years of experience, strong in SQL, Python, and Power BI or Tableau. Good communication skills are a plus."""
lang = "fr"
img_path = "1131w-uHRaEYx8dVI (1).webp"

start = datetime.now()

result = analyze_cv(cv_image_path=img_path, job_description=job, language=lang)

end = datetime.now()

elapsed = end - start
print(f"Elapsed time: {elapsed}")
print("🔍 CV Analysis Result:\n", result)

# Save to a text file
with open("cv_analysis_output_minicpm_o_2_6.txt", "w", encoding="utf-8") as file:
    file.write(result)

Elapsed time: 0:00:39.459047
🔍 CV Analysis Result:
 ### Informations extraites du CV :

1. **Nom complet :**
   - Célia Naudin

2. **Coordonnées :**
   - Téléphone : 123-456-7890
   - Email : hello@reallygreatsite.com
   - Adresse : 123 Anywhere St., Any City

3. **Formation :**
   - École Amédé Autran (2017 - 2020)
   - Master communication en alternance
   - Licence marketing digital

4. **Expérience professionnelle :**
   - **Groupe Frame** (2021 - 2026):
     - Chargée de communication:
       - Gestion de la stratégie de communication
       - Développement des réseaux sociaux
       - Production de contenus éditoriaux
       - Organisation d'événements
   - **Bancollect** (2019 - 2020):
     - Assistante communication en alternance:
       - Création et publication de contenu
       - Participation à des projets de marketing
       - Veille sur les tendances des réseaux sociaux

5. **Compétences :**
   - Réseaux sociaux
   - Graphisme
   - Référencement
   - Vidéo

### Comparaiso

### prompt 2 :

In [30]:
text = """
You are a professional AI assistant trained to analyze candidate resumes and provide evaluations strictly in the specified language.

## Your Task:
Given a job description, evaluation criteria, and a candidate's CV, analyze the resume and return a JSON object with the following structure. The explanation and all textual values (justification, strengths, weaknesses) must be written in this language: **{output_language}**.

### Output format (strictly JSON only, no extra text):
{{
  "score": 0,
  "strengths": [],
  "weaknesses": [],
  "matched_skills": [],
  "missing_skills": [],
  "final_recommendation": "",
  "justification": ""
}}

## Language:
Respond in: **{output_language}**

## Job Description:
{job_description}

## Evaluation Criteria:
{evaluation_criteria}
# Example:
# {{
#   "required_experience_years": 2,
#   "must_have_skills": ["Python", "NLP", "Transformers"],
#   "nice_to_have_skills": ["Docker", "Streamlit"],
#   "education": "Bachelor or higher",
#   "languages_required": ["English"]
# }}

IMPORTANT:
- ALL text in the JSON fields MUST be written in **{output_language}**.
- Do NOT use English. Even justification, strengths, and weaknesses must be fully translated.
- Only use {output_language} vocabulary and grammar.
"""


In [31]:
import json
from rich import print

job = """Looking for a data analyst with 3+ years of experience, strong in SQL, Python, and Power BI or Tableau. Good communication skills are a plus."""
lang = "French"

prompt = text.format(
    job_description=job,
    evaluation_criteria=json.dumps({
        "required_experience_years": 2,
        "must_have_skills": ["Python", "NLP", "Transformers"],
        "nice_to_have_skills": ["Docker", "LangChain"],
        "education": "Master",
        "languages_required": ["Français"]
    }),
    output_language=lang
)

print(prompt)


You are a professional AI assistant trained to analyze candidate resumes and provide evaluations strictly in the 
specified language.

## Your Task:
Given a job description, evaluation criteria, and a candidate's CV, analyze the resume and return a JSON object 
with the following structure. The explanation and all textual values (justification, strengths, weaknesses) must be
written in this language: **French**.

### Output format (strictly JSON only, no extra text):
{
  "score": 0,
  "strengths": [],
  "weaknesses": [],
  "matched_skills": [],
  "missing_skills": [],
  "final_recommendation": "",
  "justification": ""
}

## Language:
Respond in: **French**

## Job Description:
Looking for a data analyst with 3+ years of experience, strong in SQL, Python, and Power BI or Tableau. Good 
communication skills are a plus.

## Evaluation Criteria:
{"required_experience_years": 2, "must_have_skills": ["Python", "NLP", "Transformers"], "nice_to_have_skills": 
["Docker", "LangChain"], "education": "Master", "languages_required": ["Fran\u00e7ais"]}
# Example:
# {
#   "required_experience_years": 2,
#   "must_have_skills": ["Python", "NLP", "Transformers"],
#   "nice_to_have_skills": ["Docker", "Streamlit"],
#   "education": "Bachelor or higher",
#   "languages_required": ["English"]
# }

IMPORTANT:
- ALL text in the JSON fields MUST be written in **French**.
- Do NOT use English. Even justification, strengths, and weaknesses must be fully translated.
- Only use French vocabulary and grammar.

In [33]:
import json

def analyze_cv(cv_image_path=img_path, job_description=job, language=lang):
    
    prompt = text.format(
            job_description=job,
            evaluation_criteria=json.dumps({
                "required_experience_years": 2,
                "must_have_skills": ["Python", "NLP", "Transformers"],
                "nice_to_have_skills": ["Docker", "LangChain"],
                "education": "Master",
                "languages_required": ["Français"]
            }),
            output_language=lang
    )

        # Load image
    image = Image.open(cv_image_path).convert("RGB")

    # Build prompt
    prompt = build_cv_prompt(job_description)
    msgs = [{'role': 'user', 'content': [image, prompt]}
            ]

    response = model.chat(
        image=None,
        msgs=msgs,
        tokenizer=tokenizer,
    )


    return response


In [34]:
from datetime import datetime

start = datetime.now()



job = """Looking for a data analyst with 3+ years of experience, strong in SQL, Python, and Power BI or Tableau. Good communication skills are a plus."""
lang = "French"
img_path = "1131w-uHRaEYx8dVI (1).webp"

result = analyze_cv(cv_image_path=img_path, job_description=job, language=lang)


end = datetime.now()

elapsed = end - start
print(f"Elapsed time: {elapsed}")
print("🔍 CV Analysis Result:\n", result)

# Save to a text file
with open("cv_analysis_output_minicpm_o_2_6.txt", "w", encoding="utf-8") as file:
    file.write(result)

Elapsed time: 0:00:34.705085

🔍 CV Analysis Result:
 ### Extracted Information:

1. **Full Name**: Célia Naudin
2. **Contact Information**:
   - Email: hello@reallygreatsite.com
   - Phone: 123-456-7890
   - Address: 123 Anywhere St., Any City
3. **Education**:
   - École Amédé Autran (2017 - 2020)
   - Master in Communication en alternance
   - Licence Marketing Digital
4. **Work Experience**:
   - **Groupe Frame (2021 - 2026)**: Chargée de communication
     - Management of communication strategy
     - Development of social networks
     - Content editorial production
     - Event organization
   - **Bancollect (2019 - 2020)**: Assistant Communication en alternance
     - Content creation and publication
     - Participation in marketing projects
     - Monitoring trends on social networks
5. **Skills**:
   - Languages: French, English, Spanish
   - Competencies: Social media, Graphic design, SEO, Video
6. **Certifications/Projects**: Not specified

### Job Match Analysis:

The job description is for a data analyst with experience in SQL, Python, Power BI, or Tableau.

- **Relevance to Job Description**:
  - The resume does not mention any specific technical skills like SQL, Python, Power BI, or Tableau.
  - There are no certifications related to data analysis mentioned.

### Suitability Rating (0-100):

**Rating: 20**

### Justification:

- **Strengths**:
  - Strong background in communications and digital marketing.
  - Proficient in multiple languages including English and Spanish.
  - Demonstrated ability to manage content strategies and organize events.

- **Weaknesses**:
  - Lack of relevant technical skills such as SQL, Python, Power BI, or Tableau.
  - No evidence of education or work experience directly related to data analysis.
  - No certifications that indicate proficiency in data-related tools or methodologies.

### Summary:

**Strengths**:
- Effective communicator with strong organizational skills.
- Multilingual capabilities can be beneficial in diverse teams.
- Familiarity with graphic design and video editing could complement certain aspects of data visualization.

**Weaknesses**:
- Inadequate preparation for roles requiring advanced technical expertise in data analytics.
- Limited professional exposure to the required skill set according to the job description.

In [29]:
type(result)

str

### prompt 3 :

In [1]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional, Literal
from transformers import AutoModel, AutoTokenizer
import torch
from PIL import Image
import json
import re

# --- Pydantic Models for Structured Output ---
class PersonalInfo(BaseModel):
    full_name: str = Field(..., description="Candidate's full name")
    email: Optional[str] = Field(None, description="Contact email")
    phone: Optional[str] = Field(None, description="Phone number")
    location: Optional[str] = Field(None, description="Current location")
    linkedin: Optional[str] = Field(None, description="LinkedIn URL")
    github: Optional[str] = Field(None, description="GitHub URL")

class EducationEntry(BaseModel):
    institution: str = Field(..., description="School/university name")
    degree: Literal["bachelor", "master", "phd", "diploma", "certificate", "associate", "other"] = Field(..., description="Degree level")
    field_of_study: str = Field(..., description="Major/specialization")
    start_year: int = Field(..., description="Start year")
    end_year: Optional[int] = Field(None, description="Graduation year")

class WorkExperienceEntry(BaseModel):
    company: str = Field(..., description="Employer name")
    position: str = Field(..., description="Job title")
    start_date: str = Field(..., description="Start date (MM/YYYY)")
    end_date: Optional[str] = Field(None, description="End date (MM/YYYY or 'Present')")
    responsibilities: List[str] = Field(..., description="Key achievements")

class SkillEntry(BaseModel):
    name: str = Field(..., description="Skill name")
    category: str = Field(..., description="Skill category")
    proficiency: Literal["beginner", "intermediate", "advanced", "expert", "native"] = Field(..., description="Proficiency level")

class CVAnalysisResult(BaseModel):
    personal_info: PersonalInfo = Field(..., description="Personal details")
    education: List[EducationEntry] = Field(..., description="Education history")
    work_experience: List[WorkExperienceEntry] = Field(..., description="Work experience")
    technical_skills: List[SkillEntry] = Field(..., description="Technical skills")
    soft_skills: List[str] = Field(..., description="Soft skills")
    summary: str = Field(..., description="Professional summary")
    match_score: float = Field(..., ge=0, le=1, description="Job match score (0-1)")
    strengths: List[str] = Field(..., description="Candidate strengths for this role")
    improvement_areas: List[str] = Field(..., description="Areas needing improvement")

# --- Advanced MiniCPM-O 2.6 Integration with Language Control ---
class ProfessionalCVAnalyzer:
    SUPPORTED_LANGUAGES = ["en", "fr", "es", "de", "ar", "zh", "ja", "ru"]
    
    def __init__(self):
        self.model, self.tokenizer = self.initialize_model()
        
    def initialize_model(self):
        """Initialize MiniCPM-O 2.6 with multimodal capabilities"""
        model_id = "/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6"

        model = AutoModel.from_pretrained(
            model_id,
            trust_remote_code=True,
            attn_implementation='sdpa', # sdpa or flash_attention_2
            torch_dtype=torch.bfloat16,
            init_vision=True,
            init_audio=True,
            init_tts=True
        )
        model = model.eval().cuda()


        
        tokenizer = AutoTokenizer.from_pretrained(
            model_id,
            trust_remote_code=True
        )
        
        return model, tokenizer
    
    def create_prompt(self, job_description: str, language: str = "en") -> list:
        """Precision-engineered prompt with language control"""
        if language not in self.SUPPORTED_LANGUAGES:
            raise ValueError(f"Unsupported language. Choose from: {', '.join(self.SUPPORTED_LANGUAGES)}")
        
        schema = CVAnalysisResult.schema_json(indent=2)
        
        # Language-specific instructions
        language_instructions = {
            "en": "Output language: English",
            "fr": "Langue de sortie: Français",
            "es": "Idioma de salida: Español",
            "de": "Ausgabesprache: Deutsch",
            "ar": "لغة الإخراج: العربية",
            "zh": "输出语言: 中文",
            "ja": "出力言語: 日本語",
            "ru": "Язык вывода: Русский"
        }
        
        return [
            {
                "role": "system",
                "content": (
                    "## EXPERT PROFILE ##\n"
                    "Senior CV Analyst | Fortune 500 Recruitment Expert | Multilingual Specialist\n\n"
                    
                    "## CORE MISSION ##\n"
                    "Extract CV data and evaluate job fit with 99.8% accuracy\n\n"
                    
                    "## OPERATIONAL RULES ##\n"
                    "1. OCR EXTRACTION: Digitize all CV elements with pixel-perfect accuracy\n"
                    "2. STRUCTURED OUTPUT: Generate VALID JSON matching schema exactly\n"
                    "3. LANGUAGE CONTROL: All text output must be in specified language\n"
                    "4. JOB MATCHING: Critical evaluation against requirements\n"
                    "5. DATA NORMALIZATION:\n"
                    "   - Dates: MM/YYYY\n"
                    "   - Skills: Infer proficiency from context\n"
                    "   - Scores: Objective 0-1 scale\n"
                    "6. OUTPUT FORMAT: JSON between ```json markers\n\n"
                    
                    f"## LANGUAGE DIRECTIVE ##\n"
                    f"{language_instructions[language]}\n\n"
                    
                    f"## JOB DESCRIPTION ##\n"
                    f"{job_description}\n\n"
                    
                    f"## OUTPUT SCHEMA ##\n"
                    f"{schema}\n\n"
                    
                    "## EXECUTION PROTOCOL ##\n"
                    "1. Perform high-accuracy OCR\n"
                    "2. Extract structured data\n"
                    "3. Analyze job fit\n"
                    "4. Generate localized JSON output"
                )
            }
        ]
    
    def extract_json(self, response: str) -> dict:
        """Military-grade JSON extraction"""
        # Multi-layered extraction strategy
        patterns = [
            r'```json(.*?)```',  # Explicit JSON marker
            r'```(.*?)```',      # Generic code block
            r'\{.*\}',           # Raw JSON object
        ]
        
        for pattern in patterns:
            match = re.search(pattern, response, re.DOTALL)
            if match:
                try:
                    json_str = match.group(1).strip() if pattern != r'\{.*\}' else match.group(0)
                    # Clean non-JSON content
                    if json_str.startswith('json\n'):
                        json_str = json_str[5:]
                    return json.loads(json_str)
                except json.JSONDecodeError:
                    continue
        
        # Final fallback: AI-powered repair
        return self.ai_json_repair(response)
    
    def ai_json_repair(self, response: str) -> dict:
        """Use model intelligence to fix malformed JSON"""
        repair_prompt = [
            {"role": "user", "content": response},
            {"role": "system", "content": "Transform this text into valid JSON matching the schema. Return ONLY valid JSON."}
            
        ]
        fixed = self.model.chat(
            msgs=repair_prompt,
            tokenizer=self.tokenizer,
            #sampling=True,
            temperature=0.1
        )
        try:
            return json.loads(fixed)
        except:
            # Ultimate fallback: Extract first valid JSON
            start = fixed.find('{')
            end = fixed.rfind('}') + 1
            return json.loads(fixed[start:end])
    
    def analyze(self, cv_image_path: str, job_description: str, language: str = "en") -> CVAnalysisResult:
        """End-to-end multilingual CV analysis"""
        # Validate language
        if language not in self.SUPPORTED_LANGUAGES:
            raise ValueError(f"Unsupported language '{language}'. Valid options: {', '.join(self.SUPPORTED_LANGUAGES)}")
        
        # Load CV image
        cv_image = Image.open(cv_image_path).convert('RGB')
        
        # Prepare multimodal input with language control
        messages = self.create_prompt(job_description, language)
        messages.append({
            "role": "user",
            "content": [
                cv_image,
                "Generate professional CV analysis:"
            ]
        })
        
        # Call MiniCPM-O 2.6 with precision tuning
        response = self.model.chat(
            msgs=messages,
            tokenizer=self.tokenizer,
            sampling=True,
            temperature=0.3,  # Balance creativity and accuracy
            max_new_tokens=1800,
            top_p=0.95,
            repetition_penalty=1.1
        )
        
        # Extract and validate JSON
        try:
            json_data = self.extract_json(response)
            return CVAnalysisResult(**json_data)
        except ValidationError as e:
            # Self-healing validation system
            return self.handle_validation_error(json_data, e)
    
    def handle_validation_error(self, data: dict, error: ValidationError) -> CVAnalysisResult:
        """AI-powered schema correction"""
        error_details = str(error)
        fix_prompt = [
            {"role": "user", "content": f"Invalid JSON:\n{json.dumps(data, indent=2)}\n\nErrors:\n{error_details}"},
            {"role": "system", "content": "Correct this JSON to strictly match the schema. Return ONLY valid JSON."}
            
        ]
        fixed = self.model.chat(
            msgs=fix_prompt,
            tokenizer=self.tokenizer,
            sampling=True,
            temperature=0.1
        )
        fixed_json = self.extract_json(fixed)
        return CVAnalysisResult(**fixed_json)


In [ ]:

# --- Professional Usage Interface ---
if __name__ == "__main__":
    print("🚀 Launching Enterprise CV Analyzer v2.0")
    print("⚙️ Initializing MiniCPM-O 2.6 with multimodal capabilities...")
    analyzer = ProfessionalCVAnalyzer()
    
    job_desc = """
    Senior AI Engineer Requirements:
    - 5+ years ML production experience
    - Expertise in Python, PyTorch, TensorFlow
    - Cloud deployment (AWS/Azure/GCP)
    - PhD/MS in Computer Science
    - Publications in top AI conferences
    - Fluent English communication
    """
    
    print("\n🌍 Language Options:", ", ".join(analyzer.SUPPORTED_LANGUAGES))
    target_language = input("Select output language (default: en): ") or "en"
    
    print(f"\n🔍 Analyzing CV in {target_language.upper()}...")
    result = analyzer.analyze(
        cv_image_path="1131w-uHRaEYx8dVI (1).webp",
        job_description=job_desc,
        language=target_language
    )
    
    # Professional result presentation
    print("\n✅ ANALYSIS COMPLETE")
    print(f"📌 Candidate: {result.personal_info.full_name}")
    print(f"⭐ Match Score: {result.match_score:.0%}")
    print(f"📧 Contact: {result.personal_info.email or 'Not provided'}")
    
    print("\n🎓 Education:")
    for edu in result.education:
        print(f"- {edu.degree.capitalize()} in {edu.field_of_study}, {edu.institution} ({edu.start_year}-{edu.end_year or 'Present'})")
    
    print("\n💻 Technical Skills:")
    for skill in result.technical_skills[:5]:
        print(f"- {skill.name} ({skill.proficiency})")
    
    print("\n🌟 Key Strengths:")
    for strength in result.strengths[:3]:
        print(f"- {strength}")
    
    print("\n📈 Improvement Areas:")
    for area in result.improvement_areas:
        print(f"- {area}")
    
    print("\n💼 Professional Summary:")
    print(result.summary)

## VLLM inference :

you have to see the model exstance .bin or .safetensors

In [ ]:
!ls -l /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6

In [6]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.__version__)  # Should match your CUDA version

False
2.7.0+cpu


If False, you need to reinstall PyTorch with CUDA.

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121  # CUDA 11.8

In [ ]:
!pip uninstall vllm -y
!pip install vllm

In [ ]:
!nohup vllm serve /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6/ &

### inference :

use the following command on terminal :

1. trust the code :

nohup vllm serve /teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6/ --trust-remote-code > vllm.log 2>&1 &

2. Check the process:

bash
ps aux | grep vllm



In [ ]:
tail -n 30 nohup.out

⚡ ~ ps aux | grep vllm

3. Monitor logs:

bash
tail -f vllm.log

You should see initialization messages like:

text
INFO:     Started server process [PID]
INFO:     Waiting for application startup.

In [4]:
image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

In [5]:
vllm_model_id = "/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6"

llm_response = requests.post("http://localhost:8000/v1/completions", json={
    "model": vllm_model_id,
    "prompt": msgs,
    "max_tokens": 1000,
    "temperature": 0.3
})

print(llm_response.json())

TypeError: Object of type Image is not JSON serializable

In [8]:
import base64
from PIL import Image
import io
import requests

# Load and convert the image
image = Image.open('1131w-uHRaEYx8dVI (1).webp').convert('RGB')

# Convert image to base64
buffered = io.BytesIO()
image.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

# Format your prompt
msgs = [{
    'role': 'user',
    'content': [
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_str}"}},
        {"type": "text", "text": "What is in the image?"}
    ]
}]

vllm_model_id = "/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6"
# Send to vLLM OpenAI-compatible endpoint
response = requests.post("http://localhost:8000/v1/chat/completions", json={
    "model": vllm_model_id,
    "messages": msgs,
    "max_tokens": 1000,
    "temperature": 0.3
})

print(response.json())


{'object': 'error', 'message': 'The model `/teamspace/studios/this_studio/.cache/modelscope/hub/models/OpenBMB/MiniCPM-o-2_6` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}
